Doğrusal regresyon. Bir bölgedeki ortalama sıcaklığa, yağışa ve neme (girdi değişkenleri veya özellikler) bakarak elma ve portakal için mahsul verimini (hedef değişkenler) tahmin eden bir model oluşturacağız. İşte eğitim verileri:

https://i.imgur.com/6Ujttb4.png

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1


yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2


In [1]:
import torch as t
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Training data


In [2]:
#inputs (temp, rainfall and humidity)
inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]], dtype = 'float32')
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [3]:
#targets (apple and oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype = 'float32')
targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [4]:
# Girişleri ve hedefleri tensörlere dönüştürün
inputs = t.from_numpy(inputs)
targets = t.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch (Sıfırdan doğrusal regresyon modeli)


In [5]:
# Weights and biases (bu değerler başlangıçta random değer olmasından dolayı random fonk kullanılır)
w = t.randn(2, 3, requires_grad = True)
b = t.randn(2, requires_grad  = True)
print(w)
print(b)

tensor([[ 0.1693,  0.2874,  1.6815],
        [-0.0786,  0.7411, -0.9778]], requires_grad=True)
tensor([-1.3982,  0.0665], requires_grad=True)


Torch.randn, ortalama 0 ve standart sapma 1 ile normal bir dağılımdan rastgele seçilen elemanlarla, verilen şekle sahip bir tensör oluşturur.

In [6]:
def model(x):
    return x @ w.t() + b 
#tranposesini alma nedeni matrsi boylarını eşitleme

'@' PyTorch'ta matris çarpımını temsil eder ve '.t()' yöntemi bir tensörün tranposesini döndürür.

In [7]:
# Generate predictions
preds = model(inputs)
preds

tensor([[102.5267,   1.9359],
        [146.9228,  -4.4504],
        [149.3777,  35.8219],
        [ 90.4502, -12.2627],
        [155.5859,  -2.6596]], grad_fn=<AddBackward0>)

Modelimizin tahminlerini gerçek hedeflerle karşılaştıralım

In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Modelimizin tahminleri ile gerçek hedefler arasında büyük bir fark görebilirsiniz çünkü modelimizi rastgele ağırlıklar ve önyargılar ile başlattık. Açıkçası, rastgele başlatılmış bir modelin sadece çalışmasını bekleyemeyiz.

## Loss function



Modelimizi geliştirmeden önce, modelimizin ne kadar iyi performans gösterdiğini değerlendirmenin bir yoluna ihtiyacımız var. Aşağıdaki yöntemi kullanarak modelin tahminlerini gerçek hedeflerle karşılaştırabiliriz:

*İki matris (ön ve hedefler) arasındaki farkı hesaplayın.


*Negatif değerleri kaldırmak için fark matrisinin tüm öğelerinin karesini alın.


*Ortaya çıkan matristeki elemanların ortalamasını hesaplayın.


Sonuç, ortalama hata karesi (MSE) olarak bilinen tek bir sayıdır.

In [9]:
def mse(t1, t2):
    diff = t1 - t2
    return t.sum(diff * diff) / diff.numel()

diff.sum(), bir tensördeki tüm öğelerin toplamını döndürür. Bir tensörün diff.numel() yöntemi, tensördeki öğelerin sayısını (miktarını) döndürür. Modelimizin mevcut tahminleri için ortalama hata karesini hesaplayalım.

In [10]:
loss = mse(preds, targets)
print(loss)

tensor(5730.8018, grad_fn=<DivBackward0>)


Sonucu şu şekilde yorumlayabiliriz: Ortalama olarak, tahmindeki her bir öğe, kaybın kareköküne göre gerçek hedeften farklıdır. Ve tahmin etmeye çalıştığımız sayıların kendilerinin 50–200 aralığında olduğunu düşünürsek, bu oldukça kötü. Sonuç, modelin hedef değişkenleri tahmin etmede ne kadar kötü olduğunu gösterdiği için kayıp olarak adlandırılır. Modeldeki bilgi kaybını temsil eder: kayıp ne kadar düşükse model o kadar iyidir.

## Compute gradients

In [11]:
#compute gradients
loss.backward()

Degradeler, ilgili tensörlerin .grad özelliğinde saklanır. Kaybın türevi w.r.t. Ağırlık matrisinin kendisi aynı boyutlara sahip bir matristir.

In [12]:
print(w)
print(w.grad)

tensor([[ 0.1693,  0.2874,  1.6815],
        [-0.0786,  0.7411, -0.9778]], requires_grad=True)
tensor([[ 4529.7266,  4196.1421,  2839.0566],
        [-7287.6943, -8131.8828, -5130.1611]])


## Adjust weights and biases to reduce the loss (Kaybı azaltmak için ağırlıkları ve önyargıları ayarlayın)

Kayıp, ağırlıklarımızın ve önyargılarımızın ikinci dereceden bir fonksiyonudur ve amacımız, kaybın en düşük olduğu ağırlık setini bulmaktır. Herhangi bir ağırlık veya önyargı unsuru ile kayıp grafiğini çizersek, aşağıda gösterilen şekle benzeyecektir. Analizden elde edilen önemli bir kavrayış, gradyanın, kaybın değişim oranını, yani kayıp fonksiyonunun eğimini w.r.t gösterdiğidir. Ağırlıklar ve önyargılar.

Bir gradyan öğesi pozitifse:

--Ağırlık elemanının değerini biraz arttırmak kaybı artıracaktır


--Ağırlık elemanının değerini biraz azaltmak kaybı azaltacaktır

Bir gradyan öğesi negatifse:

--Ağırlık elemanının değerini biraz arttırmak kaybı azaltacaktır

--Ağırlık elemanının değerini biraz azaltmak, kaybı artıracaktır.

Bir ağırlık elemanının değiştirilmesiyle kayıptaki artış veya azalma, kayıp w.r.t. ile orantılıdır. Bu öğe. Bu gözlem, modelimizi geliştirmek için kullanacağımız gradyan iniş optimizasyon algoritmasının temelini oluşturur (gradyan boyunca inerek).

Her ağırlık elemanından kaybın w.r.t türevine orantılı küçük bir miktar çıkarabiliriz. Kaybı biraz azaltmak için bu unsur.

In [13]:
w
w.grad

tensor([[ 4529.7266,  4196.1421,  2839.0566],
        [-7287.6943, -8131.8828, -5130.1611]])

Ağırlıkları çok büyük miktarda değiştirmememiz için gradyanları çok küçük bir sayı (bu durumda 10 ~ -5) ile çarpıyoruz. Eğimin yokuş aşağı yönünde küçük bir adım atmak istiyoruz, dev bir sıçrama değil. Bu sayı, algoritmanın öğrenme hızı olarak adlandırılır.

1e-5 bizim öğrenme hızımızdır


t.no_grad'ı PyTorch'a ağırlıkları ve biasları güncellerken degradeleri izlemememiz, hesaplamamamız veya değiştirmememiz gerektiğini belirtmek için kullanıyoruz.

# t.no_grad()
Gradyan hesaplamasını devre dışı bırakan bağlam yöneticisi.

Gradyan hesaplamasını devre dışı bırakmak, Tensor.backward () 'ı çağırmayacağınızdan emin olduğunuzda çıkarım için kullanışlıdır. Aksi takdirde require_grad = True olacak olan hesaplamalar için bellek tüketimini azaltacaktır.

Bu modda, girişler require_grad = True olsa bile, her hesaplamanın sonucu, require_grad = False olacaktır.

Bu bağlam yöneticisi iş parçacığı yereldir; Diğer iş parçacıklarındaki hesaplamayı etkilemeyecektir.

Aynı zamanda bir dekoratör olarak da işlev görür. (Parantez ile somutlaştırdığınızdan emin olun.)

In [14]:
with t.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5 
#backward ile gradyan fonksiyonun tanımını engellemek

In [15]:
# Kaybın aslında daha düşük olduğunu doğrulayalım
loss = mse(preds, targets)
print(loss)

tensor(5730.8018, grad_fn=<DivBackward0>)


Devam etmeden önce, .zero_ () yöntemini çağırarak degradeleri sıfırlıyoruz. Bunu yapmamız gerekiyor çünkü PyTorch gradyanları biriktiriyor. Aksi takdirde, kayıpta .backward'ı bir dahaki sefere çağırdığımızda, yeni gradyan değerleri mevcut gradyanlara eklenir ve bu da beklenmedik sonuçlara yol açabilir.

In [16]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [17]:
#Gradient descent kullanarak modeli eğitin


In [18]:
#1 Generate predictions
preds = model(inputs)
#2 Calculate the loss
loss = mse(preds, targets)
#3 Compute gradients
loss.backward()
print(w.grad)
print(b.grad)
#4 Adjust weights & reset gradients
with t.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
    
print(w)
print(b)

#last calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor([[ 3773.5383,  3388.0063,  2339.2969],
        [-5947.4214, -6690.0952, -4240.6602]])
tensor([ 43.8127, -72.4196])
tensor([[ 0.0863,  0.2116,  1.6297],
        [ 0.0538,  0.8893, -0.8841]], requires_grad=True)
tensor([-1.3992,  0.0681], requires_grad=True)
tensor(2807.1777, grad_fn=<DivBackward0>)


## Eğitim için birden çok epoch 

Kaybı daha da azaltmak için, gradyanları birden çok kez kullanarak ağırlıkları ve önyargıları ayarlama işlemini tekrarlayabiliriz. Her yinelemeye epoch denir. Modeli eğitmek için 100 epoch yapalım

In [19]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with t.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

#Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(208.7466, grad_fn=<DivBackward0>)


In [20]:
# Predictions
preds

tensor([[ 59.1401,  73.0487],
        [ 90.6590,  90.9539],
        [ 96.2910, 150.6521],
        [ 34.6419,  51.7038],
        [108.4341,  93.6772]], grad_fn=<AddBackward0>)

In [21]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins (PyTorch fonksiyonlarını kullanarak lineer regresyon)



Bazı temel tensör işlemlerini kullanarak doğrusal regresyon ve gradyan iniş modeli uyguladık. Bununla birlikte, bu derin öğrenmede yaygın bir model olduğundan, PyTorch, sadece birkaç satır kodla modeller oluşturmayı ve eğitmeyi kolaylaştırmak için çeşitli yerleşik işlevler ve sınıflar sağlar.

Sinir ağları oluşturmak için yardımcı sınıflar içeren PyTorch'tan torch.nn paketini içe aktararak başlayalım.

nn (neural network)

In [22]:
import torch.nn as nn

In [23]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = t.from_numpy(inputs)
targets = t.from_numpy(targets)

In [24]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [25]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## Dataset and DataLoader


Girişlerden ve hedeflerden gelen satırlara tuple olarak erişime izin veren ve PyTorch'daki birçok farklı veri kümesiyle çalışmak için standart API'ler sağlayan bir TensorDataset oluşturacağız.

In [26]:
from torch.utils.data import TensorDataset, DataLoader

In [27]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

Ayrıca, eğitim sırasında verileri önceden tanımlanmış boyutta toplu işlere ayırabilen bir Veri Yükleyici de oluşturacağız. Ayrıca verilerin karıştırılması ve rastgele örneklenmesi gibi başka yardımcı programlar da sağlar.

In [28]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)# shuffle = karıştırma --- batch_size = yığın boyutu
#karıştırılıp rastgele örneklemler oluşturur

In [29]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  97.,  70.],
        [103.,  43.,  36.],
        [ 87., 134.,  58.],
        [ 73.,  66.,  44.],
        [ 91.,  88.,  64.]])
tensor([[102., 120.],
        [ 20.,  38.],
        [119., 133.],
        [ 57.,  69.],
        [ 81., 101.]])


Her yinelemede, veri yükleyici, verilen toplu iş boyutuna sahip bir veri grubu döndürür. Karıştırma True olarak ayarlanmışsa, gruplar oluşturmadan önce eğitim verilerini karıştırır. Karıştırma, girdiyi optimizasyon algoritmasına rastgele hale getirmeye yardımcı olarak kayıpta daha hızlı bir azalmaya yol açar.

## nn.Linear

Ağırlıkları ve biasları manuel olarak başlatmak yerine, modeli otomatik olarak yapan PyTorch'un nn.Linear sınıfını kullanarak tanımlayabiliriz.

In [30]:
model = nn.Linear(3,2) #3x2 lik matris 3 input 2 output
print(model.weight)
print("---------------------------------------------------------")
print(model.bias)

Parameter containing:
tensor([[-0.2446,  0.3400,  0.5594],
        [ 0.4791,  0.4821,  0.3623]], requires_grad=True)
---------------------------------------------------------
Parameter containing:
tensor([ 0.1449, -0.5750], requires_grad=True)


PyTorch modelleri, modelde bulunan tüm ağırlıkları ve bias matrislerini içeren bir liste döndüren yararlı bir .parameters yöntemine de sahiptir. Doğrusal regresyon modelimiz için, bir ağırlık matrisimiz ve bir bias matrisimiz var.

In [31]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2446,  0.3400,  0.5594],
         [ 0.4791,  0.4821,  0.3623]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1449, -0.5750], requires_grad=True)]

In [32]:
preds = model(inputs)
preds

tensor([[ 29.1250,  82.2766],
        [ 43.6105, 108.6316],
        [ 56.8725, 126.7164],
        [ 10.5149,  82.4283],
        [ 55.0683, 104.1208],
        [ 28.5404,  82.2737],
        [ 43.8299, 108.5118],
        [ 57.1874, 127.5578],
        [ 11.0995,  82.4312],
        [ 55.8724, 104.0039],
        [ 29.3445,  82.1568],
        [ 43.0259, 108.6287],
        [ 56.6531, 126.8363],
        [  9.7109,  82.5451],
        [ 55.6529, 104.1237]], grad_fn=<AddmmBackward>)

In [33]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## Loss Function

Bir kayıp fonksiyonunu manuel olarak tanımlamak yerine, yerleşik kayıp fonksiyonu mse_loss kullanabiliriz.

In [34]:
import torch.nn.functional as F

In [35]:
loss_fn = F.mse_loss
loss = loss_fn(preds, targets)
loss

tensor(1080.6128, grad_fn=<MseLossBackward>)

## Optimizer

Modelin ağırlıklarını ve biaslarını gradyanlar kullanarak manuel olarak değiştirmek yerine, optimize edici optim.SGD()'yi kullanabiliriz. SGD, 'stokastik gradyan iniş' in kısaltmasıdır. Stokastik terimi, numunelerin tek bir grup yerine rastgele gruplar halinde seçildiğini belirtir.

In [36]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2446,  0.3400,  0.5594],
         [ 0.4791,  0.4821,  0.3623]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1449, -0.5750], requires_grad=True)]

In [37]:
opt = t.optim.SGD(model.parameters(), lr = 1e-5)#ilk olarak model içindeki weigth ve bias değerlerini yollar


Model.parameters () öğesinin optim.SGD'ye bir argüman olarak aktarıldığını ve böylece optimize edicinin güncelleme adımı sırasında hangi matrislerin değiştirilmesi gerektiğini bildiğine dikkat edin. Ayrıca, parametrelerin değiştirilme miktarını kontrol eden bir öğrenme oranı belirleyebiliriz.

lr = 1e-5 => öğrenme parametresi

## Train the model


We are now ready to train the model. We'll follow the same process to implement gradient descent:

1--Generate predictions

2--Calculate the loss

3--Compute gradients w.r.t the weights and biases

4--Adjust the weights by subtracting a small quantity proportional to the gradient

5--Reset the gradients to zero

Tek değişiklik, her yinelemede tüm eğitim verilerini işlemek yerine veri yığınları üzerinde çalışacağımızdır. Modeli belirli sayıda dönem için eğiten bir fayda işlevi uyumu tanımlayalım.

In [38]:
def fit(num_epochs, model, loss_fonc, opt, train_ld):
    #epoch sayımızı fonktan alıyoruz
    for epoch in range(num_epochs):
        
        # Toplu veri ile eğitim (daha çok dersteki iterasyon tarzı)
        for xb,yb in train_ld:
        
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate Loss
            loss = loss_fonc(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            #bu işlem bizim elle yaptığımız güncelleme işlemi tek satırda weight ve biasları güncellememizi sağlar 
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            #teker teker w b'leri sıfırlamak yerine optimizer ile tek komutla sıfırladık
        
        if(epoch + 1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))

-Parametreleri (ağırlıklar ve önyargılar) manuel olarak güncellemek yerine, güncellemeyi gerçekleştirmek için opt.step'i ve degradeleri sıfırlamak için opt.zero_grad'ı kullanırız.

-Ayrıca, eğitimin ilerlemesini izlemek için her 10. çağ için son veri grubundaki kaybı yazdıran bir günlük ifadesi ekledik. Loss.item, kayıp tensöründe saklanan gerçek değeri döndürür

In [ ]:
fit(460, model, loss_fn, opt, train_dl)

Epoch [10/460], Loss: 76.3449
Epoch [20/460], Loss: 170.0239
Epoch [30/460], Loss: 85.3994
Epoch [40/460], Loss: 33.1973
Epoch [50/460], Loss: 54.9160
Epoch [60/460], Loss: 50.6898
Epoch [70/460], Loss: 16.4400
Epoch [80/460], Loss: 17.9817
Epoch [90/460], Loss: 24.3675
Epoch [100/460], Loss: 15.4751
Epoch [110/460], Loss: 8.2939
Epoch [120/460], Loss: 13.2757
Epoch [130/460], Loss: 11.7897
Epoch [140/460], Loss: 7.1168
Epoch [150/460], Loss: 9.5425
Epoch [160/460], Loss: 7.4617
Epoch [170/460], Loss: 5.1572
Epoch [180/460], Loss: 4.3566
Epoch [190/460], Loss: 3.2151
Epoch [200/460], Loss: 7.1410
Epoch [210/460], Loss: 5.1523


Modelimizi kullanarak tahminler oluşturalım ve hedeflerimize yakın olduklarını doğrulayalım.

In [ ]:
preds = model(inputs)
preds

In [ ]:
targets

Nitekim tahminler hedeflerimize oldukça yakın. Bir bölgedeki ortalama sıcaklık, yağış ve neme bakarak elma ve portakal için mahsul verimini tahmin etmek için oldukça iyi bir model eğittik. Tek bir girdi satırı içeren bir grup geçirerek yeni bölgeler için mahsul verimi tahminlerinde bulunmak için kullanabiliriz.

In [ ]:
model(t.tensor([[75., 63., 44.]]))

Öngörülen elma verimi hektar başına 55,4 ton ve portakalınki hektar başına 68,7 tondur.

## Machine Learning vs. Classical Programming

Bu eğitimde benimsediğimiz yaklaşım, bildiğiniz gibi programlamadan çok farklıdır. Genellikle, bazı girdiler alan, bazı işlemler gerçekleştiren ve sonuç veren programlar yazıyoruz.

Bununla birlikte, bu defterde, bazı bilinmeyen parametreler (ağırlıklar ve önyargılar) kullanılarak ifade edilen, girdiler ve çıktılar arasında belirli bir ilişki olduğunu varsayan bir 'model' tanımladık. Daha sonra modeli bazı tanıdık girdi ve çıktıları gösterir ve modeli bilinmeyen parametreler için iyi değerler üretecek şekilde eğitiriz. Eğitildikten sonra, model yeni girdiler için çıktıları hesaplamak için kullanılabilir.

Bu programlama paradigması, girdiler ve çıktılar arasındaki ilişkiyi anlamak için verileri kullandığımız makine öğrenimi olarak bilinir. Derin öğrenme, modeller oluşturmak ve eğitmek için matris işlemlerini, doğrusal olmayan etkinleştirme işlevlerini ve gradyan inişini kullanan bir makine öğrenimi dalıdır. Tesla Motors'ta AI direktörü Andrej Karpathy, bu konularda Software 2.0 başlıklı harika bir blog yazısı yazdı.

Francois Chollet'in Python ile Derin Öğrenme kitabından bu resim, klasik programlama ile makine öğrenimi arasındaki farkı yakalıyor:

*Doğrusal regresyon yerine ileri beslemeli bir sinir ağı kullanmak için, nn.Module sınıfını PyTorch'tan genişletebiliriz.

In [ ]:
import jovian
jovian.commit(project='02-linear-regression')